In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
import pickle

with open('/content/drive/MyDrive/dataset/train_dict.pkl', 'rb') as f:
    train_dict = pickle.load(f)
with open('/content/drive/MyDrive/dataset/test_dict.pkl', 'rb') as f:
    test_dict = pickle.load(f)
with open('/content/drive/MyDrive/dataset/eval_dict.pkl', 'rb') as f:
    eval_dict = pickle.load(f)

with open('/content/drive/MyDrive/dataset/train_data_processed.pkl', 'rb') as f:
    train_processed = pickle.load(f)
with open('/content/drive/MyDrive/dataset/test_data_processed.pkl', 'rb') as f:
    test_processed = pickle.load(f)
with open('/content/drive/MyDrive/dataset/eval_data_processed.pkl', 'rb') as f:
    eval_processed = pickle.load(f)

print(train_dict.keys())
print(test_dict.keys())
print(eval_dict.keys())

print(train_processed.keys())
print(test_processed.keys())
print(eval_processed.keys())


dict_keys(['id', 'title', 'context', 'question', 'answers'])
dict_keys(['id', 'title', 'context', 'question', 'answers'])
dict_keys(['id', 'title', 'context', 'question', 'answers'])
dict_keys(['input_ids', 'attention_mask', 'start_positions', 'end_positions'])
dict_keys(['input_ids', 'attention_mask', 'start_positions', 'end_positions'])
dict_keys(['input_ids', 'attention_mask', 'start_positions', 'end_positions'])


In [ ]:
from transformers import BertForQuestionAnswering

model_name = 'bert-base-uncased'
model = BertForQuestionAnswering.from_pretrained(model_name)

print(model._modules)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


OrderedDict([('bert', BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',  # output directory
    num_train_epochs=21,  # total number of training epochs, change this as you need
    per_device_train_batch_size=8,  # batch size per device during training, change this as you need
    per_device_eval_batch_size=8,  # batch size for evaluation, change this as you need
    weight_decay=0.01,  # strength of weight decay
    logging_dir='./logs',  # directory for storing logs
)


In [ ]:
from transformers import Trainer
from datasets import Dataset
import pandas as pd

train_dataset = Dataset.from_pandas(pd.DataFrame(train_processed))
eval_dataset = Dataset.from_pandas(pd.DataFrame(eval_processed))
test_dataset = Dataset.from_pandas(pd.DataFrame(test_processed))

trainer = Trainer(
    model=model,  # the instantiated Transformers model to be fine-tuned
    args=training_args,  # training arguments, defined above
    train_dataset=train_dataset,  # training dataset
    eval_dataset=eval_dataset  # evaluation dataset
)
trainer.train ()

Step,Training Loss
500,2.434200
1000,1.379900
1500,0.876600
2000,0.602100
2500,0.389000


TrainOutput(global_step=2625, training_loss=1.101282466343471, metrics={'train_runtime': 1541.0107, 'train_samples_per_second': 13.627, 'train_steps_per_second': 1.703, 'total_flos': 4115423918592000.0, 'train_loss': 1.101282466343471, 'epoch': 3.0})

In [ ]:
from transformers import Trainer
from datasets import Dataset
import pandas as pd

train_dataset = Dataset.from_pandas(pd.DataFrame(train_processed))
eval_dataset = Dataset.from_pandas(pd.DataFrame(eval_processed))
test_dataset = Dataset.from_pandas(pd.DataFrame(test_processed))

trainer = Trainer(
    model=model,  # the instantiated Transformers model to be fine-tuned
    args=training_args,  # training arguments, defined above
    train_dataset=train_dataset,  # training dataset
    eval_dataset=eval_dataset  # evaluation dataset
)
trainer.train ()

Step,Training Loss
500,0.376500
1000,0.308700
1500,0.290300
2000,0.294200
2500,0.390500
3000,0.314000
3500,0.325200
4000,0.243300
4500,0.254800
5000,0.209600


In [ ]:
from transformers import Trainer
from datasets import Dataset
import pandas as pd

train_dataset = Dataset.from_pandas(pd.DataFrame(train_processed))
eval_dataset = Dataset.from_pandas(pd.DataFrame(eval_processed))
test_dataset = Dataset.from_pandas(pd.DataFrame(test_processed))

trainer = Trainer(
    model=model,  # the instantiated Transformers model to be fine-tuned
    args=training_args,  # training arguments, defined above
    train_dataset=train_dataset,  # training dataset
    eval_dataset=eval_dataset  # evaluation dataset
)
trainer.train ("/content/drive/MyDrive/results/checkpoint-17500")

Step,Training Loss
18000,0.006600


TrainOutput(global_step=18375, training_loss=0.0002865372806990228, metrics={'train_runtime': 558.3917, 'train_samples_per_second': 263.256, 'train_steps_per_second': 32.907, 'total_flos': 2.8807967430144e+16, 'train_loss': 0.0002865372806990228, 'epoch': 21.0})

In [ ]:
def compute_exact_match(prediction, truth):
    return int(prediction == truth)

def f1_score(prediction, truth):
    pred_tokens = prediction.split()
    truth_tokens = truth.split()

    # if either the prediction or the truth is no-answer then F1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)

    common_tokens = set(pred_tokens) & set(truth_tokens)

    # if there are no common tokens then F1 = 0
    if len(common_tokens) == 0:
        return 0

    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)

    return 2 * (prec * rec) / (prec + rec)


In [ ]:
from transformers import BertTokenizer

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:

x = trainer.predict(test_dataset)
start_pos, end_pos = x.predictions
start_pos = np.argmax(start_pos, axis=1)
end_pos = np.argmax(end_pos, axis=1)


# Initialize lists to store EM and F1 scores
em_scores = []
f1_scores = []

# Iterate over predictions
for k, (i, j) in enumerate(zip(start_pos, end_pos)):
    # Get tokens for the current prediction
    tokens = tokenizer.convert_ids_to_tokens(test_processed['input_ids'][k])

    tokens = tokens.convert_tokens_to_string()
    # Join the tokens to form the predicted answer
    predicted_answer = ' '.join(tokens[i:j+1])

    # Get the correct answer from the test dictionary
    correct_answer = test_dict['answers'][k]['text'][0]  # Access the first element of the list

    # Compute EM and F1 scores
    em = compute_exact_match(predicted_answer, correct_answer)
    f1 = f1_score(predicted_answer, correct_answer)

    # Append scores to lists
    em_scores.append(em)
    f1_scores.append(f1)

    # Print results for individual prediction
    print('Question:', test_dict['question'][k])
    print('Answer:', predicted_answer)
    print('Correct Answer:', correct_answer)
    print('Exact Match:', em)
    print('F1 Score:', f1)
    print('---')

# Calculate average and median scores
avg_em = np.mean(em_scores)
median_em = np.median(em_scores)
avg_f1 = np.mean(f1_scores)
median_f1 = np.median(f1_scores)

# Print average and median scores
print('Average EM:', avg_em)
print('Median EM:', median_em)
print('Average F1 Score:', avg_f1)
print('Median F1 Score:', median_f1)


Streaming output truncated to the last 5000 lines.
Correct Answer: some forms of securitization
Exact Match: 0
F1 Score: 0
---
Question: How many more text votes were there in season eight of American Idol over season seven? 
Answer: 100 million
Correct Answer: 100 million
Exact Match: 1
F1 Score: 1.0
---
Question: In what borough is the garment business prominent?
Answer: brooklyn
Correct Answer: Brooklyn
Exact Match: 0
F1 Score: 0
---
Question: What was the most prominent religion in New York as of 2014?
Answer: judaism
Correct Answer: Christianity
Exact Match: 0
F1 Score: 0
---
Question: Which season five contestant caused controversy with their performance of I Walk the Line?
Answer: chris da ##ught ##ry
Correct Answer: Chris Daughtry
Exact Match: 0
F1 Score: 0
---
Question: By which language was the Astur-Leonese replaced?
Answer: cast ##ilia ##n
Correct Answer: Castilian (Spanish Language)
Exact Match: 0
F1 Score: 0
---
Question: Which judge joined the panel?
Answer: kara di ##og

In [ ]:

x = trainer.predict(test_dataset)
start_pos, end_pos = x.predictions
start_pos = np.argmax(start_pos, axis=1)
end_pos = np.argmax(end_pos, axis=1)

# Initialize lists to store EM and F1 scores
em_scores = []
f1_scores = []

# Iterate over predictions
for k, (i, j) in enumerate(zip(start_pos, end_pos)):
    # Get tokens for the current prediction
    tokens = tokenizer.convert_ids_to_tokens(test_processed['input_ids'][k])

    # Convert token list to string
    predicted_answer = tokenizer.convert_tokens_to_string(tokens[i:j+1])

    # Preprocess the predicted and correct answers
    predicted_answer = predicted_answer.lower()
    correct_answer = test_dict['answers'][k]['text'][0].lower()

    # Compute EM and F1 scores
    em = compute_exact_match(predicted_answer, correct_answer)
    f1 = f1_score(predicted_answer, correct_answer)

    # Append scores to lists
    em_scores.append(em)
    f1_scores.append(f1)

    # Print results for individual prediction
    print('Question:', test_dict['question'][k])
    print('Answer:', predicted_answer)
    print('Correct Answer:', correct_answer)
    print('Exact Match:', em)
    print('F1 Score:', f1)
    print('---')

# Calculate average and median scores
avg_em = np.mean(em_scores)
median_em = np.median(em_scores)
avg_f1 = np.mean(f1_scores)
median_f1 = np.median(f1_scores)

# Print average and median scores
print('Average EM:', avg_em)
print('Median EM:', median_em)
print('Average F1 Score:', avg_f1)
print('Median F1 Score:', median_f1)


In [ ]:
from transformers import pipeline
import numpy as np

question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')

for i in range(len(test_dict['question'][:2])):
    result = question_answerer(question=test_dict['question'][i], context=test_dict['context'][i])
    print('Question:', test_dict['question'][i])
    print('Answer:', result['answer'])
    print('Correct Answer:', test_dict['answers'][i]['text'][0])
    print('Exact Match:', compute_exact_match(result['answer'], test_dict['answers'][i]['text'][0]))
    print('F1 Score:', f1_score(result['answer'], test_dict['answers'][i]['text'][0]))
    print('---')


# Calculate average and median scores
avg_em = np.mean(em_scores)
median_em = np.median(em_scores)
avg_f1 = np.mean(f1_scores)
median_f1 = np.median(f1_scores)

# Print average and median scores
print('Average EM:', avg_em)
print('Median EM:', median_em)
print('Average F1 Score:', avg_f1)
print('Median F1 Score:', median_f1)



config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Question: Along with Staten Island and the Bronx, what borough is served by the New York Public Library?
Answer: Manhattan
Correct Answer: Manhattan
Exact Match: 1
F1 Score: 1.0
---
Question: Who are some notable musical composers from Portugal?
Answer: José Vianna da Motta, Carlos Seixas
Correct Answer: José Vianna da Motta, Carlos Seixas, João Domingos Bomtempo, João de Sousa Carvalho, Luís de Freitas Branco and his student Joly Braga Santos
Exact Match: 0
F1 Score: 0.3448275862068966
---
Average EM: 0.002
Median EM: 0.0
Average F1 Score: 0.01933708597777827
Median F1 Score: 0.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import pipeline
import numpy as np
# Initialize question answering pipeline
question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')

# Initialize lists to store EM and F1 scores
em_scores1 = []
f1_scores1 = []

# Iterate over test questions
for i in range(len(test_dict['question'][:20])):
    # Get prediction from question answering model
    result = question_answerer(question=test_dict['question'][i], context=test_dict['context'][i])

    # Compute EM and F1 scores for the prediction
    em = compute_exact_match(result['answer'], test_dict['answers'][i]['text'][0])
    f1 = f1_score(result['answer'], test_dict['answers'][i]['text'][0])

    # Append scores to lists
    em_scores1.append(em)
    f1_scores1.append(f1)

    # Print results for individual question
    print('Question:', test_dict['question'][i])
    print('Answer:', result['answer'])
    print('Correct Answer:', test_dict['answers'][i]['text'][0])
    print('Exact Match:', em)
    print('F1 Score:', f1)
    print('---')

# Calculate average and median scores
avg_em = np.mean(em_scores1)
median_em = np.median(em_scores1)
avg_f1 = np.mean(f1_scores1)
median_f1 = np.median(f1_scores1)

# Print average and median scores
print('Average EM:', avg_em)
print('Median EM:', median_em)
print('Average F1 Score:', avg_f1)
print('Median F1 Score:', median_f1)


Question: Along with Staten Island and the Bronx, what borough is served by the New York Public Library?
Answer: Manhattan
Correct Answer: Manhattan
Exact Match: 1
F1 Score: 1.0
---
Question: Who are some notable musical composers from Portugal?
Answer: José Vianna da Motta, Carlos Seixas
Correct Answer: José Vianna da Motta, Carlos Seixas, João Domingos Bomtempo, João de Sousa Carvalho, Luís de Freitas Branco and his student Joly Braga Santos
Exact Match: 0
F1 Score: 0.3448275862068966
---
Question: How many miles was the village Frédéric born in located to the west of Warsaw?
Answer: 29
Correct Answer: 29
Exact Match: 1
F1 Score: 1.0
---
Question: How many attendants accompanied the flame during it's travels?
Answer: 30
Correct Answer: 30
Exact Match: 1
F1 Score: 1.0
---
Question: What artist was Kanye's third album release competing against?
Answer: 50 Cent
Correct Answer: 50 Cent
Exact Match: 1
F1 Score: 1.0
---
Question: In what year did Chopin and Sand ultimately bring their rela